Attention la j'ai fait la rasterisation de tout le vecteur pas seulement de la classif pixel

In [1]:
# Importation des librairies
from my_function import ( 
    train_random_forest, 
    save_classification,
    sel_classif_pixel,
    supprimer_dossier_non_vide)
import os
import geopandas as gpd
import pandas as pd
import sys
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw
# import classification as cla

In [2]:
# Création du dossier de sauvegarde temporaire
output_dir = "/home/onyxia/work/output"
os.makedirs(output_dir, exist_ok=True) # Création d'un dossier output temporaire

In [3]:
# Save d'un vecteur echantillons avec que les données pour classif pixel
sample_p = "/home/onyxia/work/Depot_Git/results/data/sample/Sample_BD_foret_T31TCJ.shp"
sample = gpd.read_file(sample_p)
sample_px = sel_classif_pixel(sample)
sample_px_file = os.path.join(output_dir,"sample_classif_px.shp")
sample_px.to_file(sample_px_file)

In [4]:
# cmd rasterisation paramètres
in_vector = '/home/onyxia/work/output/sample_classif_px.shp'
ref_image = '/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_allbands.tif'
ref_image_open = rw.open_image(ref_image)
out_image = os.path.join(output_dir, "rasterized_sample.tif")
field_name = 'Code'  # field containing the numeric label of the classes
sptial_resolution = rw.get_pixel_size(ref_image_open)[0]
xmin,ymax = rw.get_origin_coordinates(ref_image_open)
y,x = rw.get_image_dimension(ref_image_open)[0:2]
xmax,ymin = xmin+x*10,ymax-y*10

# define command pattern to fill with paremeters
cmd_pattern = ("gdal_rasterize -a {field_name} "
               "-tr {sptial_resolution} {sptial_resolution} "
               "-te {xmin} {ymin} {xmax} {ymax} -ot Byte -of GTiff "
               "{in_vector} {out_image}")

# fill the string with the parameter thanks to format function
cmd = cmd_pattern.format(in_vector=in_vector, xmin=xmin, ymin=ymin, xmax=xmax,
                         ymax=ymax, out_image=out_image, field_name=field_name,
                         sptial_resolution=sptial_resolution)

# execute the command in the terminal
os.system(cmd)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [ ]:
# Entraîner le modèle
model = train_random_forest(Samples_X, Class_Y)

In [ ]:
# Sauvegarder la carte
save_classification(model, images, 'results/data/classif/carte_essences_echelle_pixel.tif')

In [ ]:
## Nettoyage des dossiers
supprimer_dossier_non_vide(output_dir)